In [7]:
!pip install cvzone

  Using cached cvzone-1.5.6-py3-none-any.whl


In [3]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

# For webcam input:
cap = cv2.VideoCapture(1)

with mp_hands.Hands(min_detection_confidence=0.5, min_tracking_confidence=0.5) as hands:
    while cap.isOpened():
        success, image = cap.read()
        if not success:
            print("Ignoring empty camera frame.")
            # If loading a video, use 'break' instead of 'continue'.
            continue

        # Flip the image horizontally for a later selfie-view display, and convert
        # the BGR image to RGB.
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        # To improve performance, optionally mark the image as not writeable to
        # pass by reference.
        image.flags.writeable = False
        results = hands.process(image)

        # Draw the hand annotations on the image.
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                x, y = hand_landmarks.landmark[9].x, hand_landmarks.landmark[9].y
                x1, y1 = hand_landmarks.landmark[12].x, hand_landmarks.landmark[12].y

                cv2.circle(image, (int(x), int(y)), 10, (0, 255, 0), -1)
                cv2.circle(image, (int(x1), int(y1)), 10, (0, 0, 255), -1)

                if y1 > y:
                    hand_status = "closed"

                else:
                    hand_status = "open"
                cv2.putText(image, hand_status, (50, 50), 0, 1.5, (0, 0, 255), 2)

        cv2.imshow('MediaPipe Hands', image)
        if cv2.waitKey(5) & 0xFF == 27:
            break
cap.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

In [83]:
import cv2
import cvzone
from cvzone.HandTrackingModule import HandDetector
from directkeys import PressKey, ReleaseKey
from directkeys import space_pressed
import time
from ctypes import *

detector = HandDetector(detectionCon = 0.8, maxHands = 1)

space_key_pressed = space_pressed

time.sleep(2.0)

current_key_pressed = set()

video = cv2.VideoCapture(1)

while True:
    ret, frame = video.read()
    
    keyPressed = False
    spacePressed = False
    key_count = 0
    key_pressed = 0
    
    hands ,img = detector.findHands(frame)
    cv2.rectangle(img, (1900, 1050), (1440, 980), (100, 200, 100), -2)
    cv2.rectangle(img, (10, 1050), (565, 980), (100, 200, 100), -2)

    if hands:
        lmList = hands[0]
        fingerUp = detector.fingersUp(lmList)
#         print(fingerUp)
        
        # Finger Count
        if fingerUp == [0, 0, 0, 0, 0]:
            cv2.putText(frame, 'Finger Count: 0', (20, 1030), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(frame, 'Jumping', (1600, 1030), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
            
            PressKey(space_key_pressed)
            spacePressed = True
            current_key_pressed.add(space_key_pressed)
            key_pressed = space_key_pressed
            keyPressed = True
            key_count = key_count + 1
        
        if fingerUp == [0, 1, 0, 0, 0]:
            cv2.putText(frame, 'Finger Count: 1', (20, 1030), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(frame, 'Not Jumping', (1450, 1030), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        if fingerUp == [0, 1, 1, 0, 0]:
            cv2.putText(frame, 'Finger Count: 2', (20, 1030), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(frame, 'Not Jumping', (1450, 1030), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        if fingerUp == [0, 1, 1, 1, 0]:
            cv2.putText(frame, 'Finger Count: 3', (20, 1030), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(frame, 'Not Jumping', (1450, 1030), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        if fingerUp == [0, 1, 1, 1, 1]:
            cv2.putText(frame, 'Finger Count: 4', (20, 1030), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(frame, 'Not Jumping', (1450, 1030), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        if fingerUp == [1, 1, 1, 1, 1]:
            cv2.putText(frame, 'Finger Count: 5', (20, 1030), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)    
            cv2.putText(frame, 'Not Jumping', (1450, 1030), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        
        if not keyPressed and len(current_key_pressed) != 0:
            for key in current_key_pressed:
                ReleaseKey(key)
            current_key_pressed = set()
        elif key_count == 1 and len(current_key_pressed) ==2:
            for key in current_key_pressed:
                if key_pressed != key:
                    ReleaseKey(key)
            current_key_pressed = set()
            for key in current_key_pressed:
                ReleaseKey(key)
            current_key_pressed = set()
            
            
#     frame = cv2.flip(frame, 1)
    cv2.imshow('Frame', frame)
    k = cv2.waitKey(1)
    if k == ord('q'):
        break

video.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

AttributeError: module 'ctypes' has no attribute 'windll'

In [84]:
import cv2
from cvzone.HandTrackingModule import HandDetector
from directkeys import PressKey, ReleaseKey
from directkeys import space_pressed
import time

detector=HandDetector(detectionCon=0.8, maxHands=1)

space_key_pressed=space_pressed

time.sleep(2.0)

current_key_pressed = set()

video=cv2.VideoCapture(0)

while True:
    ret,frame=video.read()
    keyPressed = False
    spacePressed=False
    key_count=0
    key_pressed=0   
    hands,img=detector.findHands(frame)
    cv2.rectangle(img, (1900, 1050), (1440, 980), (100, 200, 100), -2)
    cv2.rectangle(img, (10, 1050), (565, 980), (100, 200, 100), -2)
    if hands:
        lmList=hands[0]
        fingerUp=detector.fingersUp(lmList)
        print(fingerUp)
        if fingerUp==[0,0,0,0,0]:
            cv2.putText(frame, 'Finger Count: 0', (20, 1030), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(frame, 'Jumping', (1600, 1030), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
            PressKey(space_key_pressed)
            spacePressed=True
            current_key_pressed.add(space_key_pressed)
            key_pressed=space_key_pressed
            keyPressed = True
            key_count=key_count+1
        if fingerUp == [0, 1, 0, 0, 0]:
            cv2.putText(frame, 'Finger Count: 1', (20, 1030), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(frame, 'Not Jumping', (1450, 1030), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        if fingerUp == [0, 1, 1, 0, 0]:
            cv2.putText(frame, 'Finger Count: 2', (20, 1030), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(frame, 'Not Jumping', (1450, 1030), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        if fingerUp == [0, 1, 1, 1, 0]:
            cv2.putText(frame, 'Finger Count: 3', (20, 1030), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(frame, 'Not Jumping', (1450, 1030), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        if fingerUp == [0, 1, 1, 1, 1]:
            cv2.putText(frame, 'Finger Count: 4', (20, 1030), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(frame, 'Not Jumping', (1450, 1030), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        if fingerUp == [1, 1, 1, 1, 1]:
            cv2.putText(frame, 'Finger Count: 5', (20, 1030), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)    
            cv2.putText(frame, 'Not Jumping', (1450, 1030), cv2.FONT_HERSHEY_COMPLEX, 2, (255, 255, 255), 2, cv2.LINE_AA)
        if not keyPressed and len(current_key_pressed) != 0:
            for key in current_key_pressed:
                ReleaseKey(key)
            current_key_pressed = set()
        elif key_count==1 and len(current_key_pressed)==2:    
            for key in current_key_pressed:             
                if key_pressed!=key:
                    ReleaseKey(key)
            current_key_pressed = set()
            for key in current_key_pressed:
                ReleaseKey(key)
            current_key_pressed = set()
    cv2.imshow("Frame",frame)
    k=cv2.waitKey(1)
    if k==ord('q'):
        break

video.release()
cv2.destroyAllWindows()
cv2.waitKey(1)

AttributeError: module 'ctypes' has no attribute 'windll'